In [1]:
import json

In [2]:
with open('/Users/etashguha/Downloads/final_mle_dataset.json', 'r') as f:
    data = json.load(f)

In [3]:
print(len(data))

1017


In [5]:
data[0].keys()

dict_keys(['product_name', 'product_description', 'prospect_name', 'prospect_industry', 'prospect_title', 'email', 'accepted', 'critique', 'edited', 'email_embedding', 'edited_embedding'])

In [9]:
accepted = [obj for obj in data if obj.get('accepted') == 1]
rejected = [obj for obj in data if obj.get('accepted') == 0]

In [10]:
len(accepted)

524

In [11]:
len(rejected)

493

In [31]:
def separate_json_by_keys(json_data, keys):
    separated_jsons = {}
    for obj in json_data:
        key_values = tuple(obj[key] for key in keys)
        if key_values not in separated_jsons:
            separated_jsons[key_values] = []
        separated_jsons[key_values].append(obj)
    return separated_jsons
sort_key_list = ['prospect_industry']

sorted_jsons = separate_json_by_keys(data, sort_key_list)

In [91]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

def remove_fillers(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Get the list of English stopwords
    stop_words = set(stopwords.words('english'))

    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize and filter out stopwords from the tokens
    filtered_tokens = [lemmatizer.lemmatize(token.lower(), pos='v') for token in tokens if token.lower() not in stop_words]

    # Join the filtered tokens back into a string
    filtered_text = ' '.join(filtered_tokens)

    return filtered_text

[nltk_data] Downloading package punkt to /Users/etashguha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/etashguha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
import re
def preprocess_text(text):
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)

    # Capitalize the text
    text = text.upper()

    return text
critical_words = []
changed_words = []
removed_words = []
for reject in rejected:
    critical_words.extend(preprocess_text(remove_fillers(reject["critique"])).split(" "))
    words_in_original_email = set(preprocess_text(remove_fillers(reject["email"])).split(" "))
    words_in_new_email = set(preprocess_text(remove_fillers(reject["edited"])).split(" "))
    changed_words.extend(list(words_in_new_email.difference(words_in_original_email)))
    removed_words.extend(list(words_in_original_email.difference(words_in_new_email)))  

In [105]:
from collections import Counter

def find_most_common(lst, n):
    counter = Counter(lst)
    most_common = counter.most_common(n)
    return most_common
# find_most_common(critical_words, 100) 
rejected[300]["critique"]
# find_most_common(changed_words, 100) 
# find_most_common(removed_words, 100) 

"The email is too generic and lacks personalization. It doesn't establish a connection with the prospect or offer any value proposition. The use of 'unbeatable prices' may diminish the perceived value of the product."

In [99]:
import copy
accepted_names = []

def find_best_and_worst_categories(key):
    for accept in accepted:
        accepted_names.extend(remove_fillers(preprocess_text(accept[key])).split(" "))
    rejected_names = []
    for reject in rejected:
        rejected_names.extend(remove_fillers(preprocess_text(reject[key])).split(" "))
    good_names = copy.deepcopy(accepted_names)
    for bad_name in rejected_names:
        if bad_name in good_names:
            good_names.remove(bad_name)
    bad_names = copy.deepcopy(rejected_names)
    for good_name in accepted_names:
        if good_name in bad_names:
            bad_names.remove(good_name)
    print(find_most_common(good_names, 100))
    print(find_most_common(bad_names, 100))

find_best_and_worst_categories("prospect_industry")

[('entertainment', 19), ('fitness', 13), ('travel', 11), ('game', 10), ('education', 8), ('real', 7), ('estate', 7), ('beauty', 6), ('energy', 6), ('environmental', 4), ('arts', 4), ('transportation', 3), ('interior', 3), ('design', 3), ('architecture', 2), ('clean', 2), ('electrical', 2), ('art', 2), ('ecommerce', 2), ('agriculture', 2), ('law', 2), ('finance', 2), ('environment', 1), ('parent', 1), ('animal', 1), ('welfare', 1), ('', 1), ('dentistry', 1), ('care', 1), ('childcare', 1), ('tourism', 1), ('music', 1), ('landscape', 1), ('enforcement', 1), ('veterinary', 1), ('automotive', 1), ('renewable', 1), ('health', 1)]
[('healthcare', 19), ('hospitality', 12), ('sport', 9), ('beverage', 8), ('food', 8), ('business', 5), ('fashion', 5), ('service', 4), ('none', 4), ('technology', 4), ('pediatrics', 3), ('engineer', 2), ('event', 2), ('plan', 2), ('market', 2), ('security', 2), ('construction', 2), ('photography', 1), ('publish', 1), ('manufacture', 1), ('advertise', 1), ('appliance